In [33]:
import os
import imutils
import cv2
import numpy as np
import joblib
from scipy.cluster.vq import *

In [34]:
# 加载已训练的模型
clf, classes_names, stdSlr, k, voc = joblib.load("classfication.pkl")
# 特征提取和train.py类似
sift = cv2.SIFT_create()
def predict_image(image_path):
    # List where all the descriptors are stored
    des_list = []
    im = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    im = cv2.resize(im, (300, 300))
    kpts = sift.detect(im)
    kpts, des = sift.compute(im, kpts)
    des_list.append((image_path, des))
    descriptors = des_list[0][1]
    for image_path, descriptor in des_list[0:]:
        descriptors = np.vstack((descriptors, descriptor))
    test_features = np.zeros((1, k), "float32")
    words, distance = vq(des_list[0][1], voc)
    for w in words:
        test_features[0][w] += 1
    # Perform Tf-Idf vectorization
    nbr_occurences = np.sum((test_features > 0) * 1, axis=0)
    idf = np.array(np.log((1.0 + 1) / (1.0 * nbr_occurences + 1)), 'float32')
    # Scale the features
    test_features = stdSlr.transform(test_features)
    # Perform the predictions
    predictions = [classes_names[i] for i in clf.predict(test_features)]
    return predictions
if __name__ == "__main__":
    test_path = "Dataset/test/"
    testing_names = os.listdir(test_path)
    image_paths = []
    for training_name in testing_names:
        dir = os.path.join(test_path, training_name)
        class_path = [os.path.join(dir, f) for f in os.listdir(dir)]
        image_paths += class_path
    bingo = 1.0
    for image_path in image_paths:
        predictions = predict_image(image_path)
        print("image: %s, classes : %s"%(image_path, predictions))
        if '.'.join(predictions) in image_path:
            bingo += 1
    print("准确率:", bingo / len(image_paths))

image: Dataset/test/chicken/ILSVRC2012_val_00003606.JPEG, classes : ['chicken']
image: Dataset/test/chicken/ILSVRC2012_val_00000911.JPEG, classes : ['chicken']
image: Dataset/test/chicken/ILSVRC2012_val_00003605.JPEG, classes : ['fish']
image: Dataset/test/chicken/ILSVRC2012_val_00004306.JPEG, classes : ['bird']
image: Dataset/test/chicken/ILSVRC2012_val_00004463.JPEG, classes : ['chicken']
image: Dataset/test/chicken/ILSVRC2012_val_00000329.JPEG, classes : ['dog']
image: Dataset/test/dog/ILSVRC2012_val_00002150.JPEG, classes : ['bird']
image: Dataset/test/dog/ILSVRC2012_val_00002288.JPEG, classes : ['dog']
image: Dataset/test/dog/ILSVRC2012_val_00001164.JPEG, classes : ['bird']
image: Dataset/test/dog/ILSVRC2012_val_00001141.JPEG, classes : ['dog']
image: Dataset/test/dog/ILSVRC2012_val_00000613.JPEG, classes : ['dog']
image: Dataset/test/dog/ILSVRC2012_val_00001848.JPEG, classes : ['dog']
image: Dataset/test/fish/ILSVRC2012_val_00018075.JPEG, classes : ['chicken']
image: Dataset/test